# Extração das planilhas

In [7]:
import os
from glob import glob

# Buscar todos os arquivos Excel (.xls) na pasta "Planilhas"
file_paths = glob(os.path.join(pasta, '*.xls'))

for fpath in file_paths:
    try:
        # Carregar o arquivo Excel
        xls = pd.ExcelFile(fpath)
        
        # Verificar se existe a planilha "Estoques_2000-2020"
        if "Estoques_2000-2020" in xls.sheet_names:
            df_sheet = xls.parse("Estoques_2000-2020")
            
            # Gerar nome para o CSV de saída
            base_name = os.path.splitext(os.path.basename(fpath))[0]
            csv_path = os.path.join(pasta, f"{base_name}_Estoques_2000-2020.csv")
            
            # Salvar a planilha em CSV
            df_sheet.to_csv(csv_path, index=False)
            print(f"Arquivo convertido: {csv_path}")
        else:
            print(f"Planilha 'Estoques_2000-2020' não encontrada em {fpath}")
    except Exception as e:
        print(f"Erro ao processar {fpath}: {e}")

Arquivo convertido: Planilhas\AC_Mudancas_Estoques_00_10_12_14_16_18_20_serie_revisada_Estoques_2000-2020.csv
Arquivo convertido: Planilhas\AL_Mudancas_Estoques_00_10_12_14_16_18_20_serie_revisada_Estoques_2000-2020.csv
Arquivo convertido: Planilhas\AM_Mudancas_Estoques_00_10_12_14_16_18_20_serie_revisada_Estoques_2000-2020.csv
Arquivo convertido: Planilhas\AP_Mudancas_Estoques_00_10_12_14_16_18_20_serie_revisada_Estoques_2000-2020.csv
Arquivo convertido: Planilhas\BA_Mudancas_Estoques_00_10_12_14_16_18_20_serie_revisada_Estoques_2000-2020.csv
Arquivo convertido: Planilhas\Brasil_Mudancas_Estoques_00_10_12_14_16_18_20_serie_revisada_Estoques_2000-2020.csv
Arquivo convertido: Planilhas\CE_Mudancas_Estoques_00_10_12_14_16_18_20_serie_revisada_Estoques_2000-2020.csv
Arquivo convertido: Planilhas\DF_Mudancas_Estoques_00_10_12_14_16_18_20_serie_revisada_Estoques_2000-2020.csv
Arquivo convertido: Planilhas\ES_Mudancas_Estoques_00_10_12_14_16_18_20_serie_revisada_Estoques_2000-2020.csv
Arquiv

# Extração das variáveis

In [8]:
import pandas as pd
import os
import re

# Caminho da pasta com os arquivos .xls convertidos para .csv
pasta = "CSV"

# Categorias relevantes para fuzzy multiobjetivo
categorias_fuzzy = [
    "Área Agrícola",
    "Pastagem com Manejo",
    "Vegetação Florestal",
    "Mosaico de Ocupações em Área Florestal",
    "Mosaico de Ocupações em Área Campestre",
    "Área Descoberta",
    "Área Artificial",
    "Área Úmida"
]

# Lista para guardar os dados de todos os estados
registros = []

# Percorre cada arquivo na pasta
for arquivo in os.listdir(pasta):
    if arquivo.endswith(".csv"):
        caminho_arquivo = os.path.join(pasta, arquivo)
        estado = arquivo[:2].upper()  # extrair a sigla do estado do nome do arquivo (ex: AC, BA)

        try:
            df = pd.read_csv(caminho_arquivo)
            df.rename(columns={df.columns[0]: "Indicador"}, inplace=True)

            # Identificar linha das categorias
            linha_categorias_index = df[df["Indicador"].astype(str).str.contains("Estoque", case=False, na=False)].index[0] - 1
            categorias = df.iloc[linha_categorias_index, 1:].values

            # Filtrar linhas válidas (Estoque com ano)
            linhas_estoque = df[df["Indicador"].str.contains(r"Estoque \(\d{4}\)", na=False, regex=True)]

            for _, row in linhas_estoque.iterrows():
                ano = re.findall(r"\d{4}", row["Indicador"])[0]
                valores = row[1:].values

                linha_dict = {"Estado": estado, "Ano": int(ano)}
                for cat, val in zip(categorias, valores):
                    if isinstance(cat, str) and cat.strip() in categorias_fuzzy:
                        linha_dict[cat.strip()] = val
                registros.append(linha_dict)

        except Exception as e:
            print(f"Erro ao processar {arquivo}: {e}")

# Criar DataFrame final
df_final = pd.DataFrame(registros)

# Ordenar colunas
colunas_ordenadas = ["Estado", "Ano"] + [cat for cat in categorias_fuzzy if cat in df_final.columns]
df_final = df_final[colunas_ordenadas]
df_final = df_final.sort_values(by=["Estado", "Ano"])

# Salvar como CSV limpo e pronto para fuzzy MO
df_final.to_csv("estoques_fuzzy_todos_estados.csv", index=False)
print("✅ CSV salvo como estoques_fuzzy_todos_estados.csv")


✅ CSV salvo como estoques_fuzzy_todos_estados.csv


# Normalizar INPC

In [22]:
import pandas as pd

# Passo 1: Ler o arquivo manualmente
with open("INPC.txt", "r", encoding="utf-8") as f:
    linhas = f.readlines()

# Supomos que a primeira linha seja o cabeçalho separado por vírgula
cabecalho = linhas[0].strip().split(",")
dados = []
for linha in linhas[1:]:
    # Usar split() (que separa por espaços e tabulações)
    partes = linha.strip().split()
    if len(partes) >= 2:
        dados.append(partes[:2])

# Criar o DataFrame
df = pd.DataFrame(dados, columns=cabecalho)

# Passo 2: Converter a coluna "Valor" – trocar vírgula por ponto e converter para float
df["Valor"] = df["Valor"].astype(str).str.replace(",", ".").astype(float)

# Passo 3: Converter a coluna "Data" para datetime
# Mapeamento dos meses de português para número
month_map_num = {
    'jan': '01', 'fev': '02', 'mar': '03', 'abr': '04', 'mai': '05',
    'jun': '06', 'jul': '07', 'ago': '08', 'set': '09', 'out': '10',
    'nov': '11', 'dez': '12'
}

def convert_date(date_str):
    """
    Converte uma data no formato 'abr/79' para um datetime, assumindo o dia 01.
    Se o ano for >= 50, assume 1900+ano; caso contrário, 2000+ano.
    """
    date_str = date_str.strip()
    partes = date_str.split("/")
    if len(partes) != 2:
        return pd.NaT
    mes, ano = partes[0].strip(), partes[1].strip()
    mes_num = month_map_num.get(mes.lower())
    if not mes_num:
        return pd.NaT
    try:
        ano_int = int(ano)
    except:
        return pd.NaT
    full_year = 1900 + ano_int if ano_int >= 50 else 2000 + ano_int
    data_formatada = f"01/{mes_num}/{full_year}"
    return pd.to_datetime(data_formatada, format="%d/%m/%Y", errors="coerce")

df["Data"] = df["Data"].apply(convert_date)

# Remover linhas onde a conversão falhou
df = df.dropna(subset=["Data"])

# Passo 4: Extrair o ano
df["Ano"] = df["Data"].dt.year

# Passo 5: Calcular a média do INPC por ano
df_anual = df.groupby("Ano")["Valor"].mean().reset_index()
df_anual.rename(columns={"Valor": "INPC_Medio"}, inplace=True)

# Exibir os anos disponíveis para diagnóstico
print("Anos disponíveis:", df_anual["Ano"].tolist())

# Passo 6: Filtrar os anos que interessam (por exemplo, os anos dos estoques)
anos_estoques = [2000, 2010, 2012, 2014, 2016, 2018, 2020]
df_alinhado = df_anual[df_anual["Ano"].isin(anos_estoques)]

# Se não houver nenhum ano de interesse, salva todos os anos (para diagnóstico)
if df_alinhado.empty:
    print("Nenhum dos anos especificados foi encontrado; salvando todos os anos disponíveis.")
    df_alinhado = df_anual

# Passo 7: Salvar o resultado em CSV
df_alinhado.to_csv("inpc_alinhado_com_estoques.csv", index=False)
print("✅ Arquivo gerado: inpc_alinhado_com_estoques.csv")
print(df_alinhado)


Anos disponíveis: [1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
✅ Arquivo gerado: inpc_alinhado_com_estoques.csv
     Ano  INPC_Medio
21  2000    0.430000
31  2010    0.524167
33  2012    0.502500
35  2014    0.505000
37  2016    0.533333
39  2018    0.282500
41  2020    0.444167


# Normalização e Calculo das funções Fuzzy

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Carregar a base de dados dos estoques de uso da terra
# O arquivo deve conter as colunas "Estado", "Ano" e as variáveis dos estoques
df = pd.read_csv("estoques_fuzzy_todos_estados.csv")  # Substitua pelo nome do seu arquivo

# Definir os critérios relevantes para o modelo fuzzy multiobjetivo
criterios = [
    "Área Agrícola",
    "Pastagem com Manejo",
    "Vegetação Florestal",
    "Mosaico de Ocupações em Área Florestal",
    "Mosaico de Ocupações em Área Campestre",
    "Área Descoberta",
    "Área Artificial",
    "Área Úmida"
]

# Definir o objetivo para cada critério:
# "max" indica que queremos maximizar o valor (quanto maior, melhor)
# "min" indica que queremos minimizar o valor (quanto menor, melhor)
objetivos = {
    "Área Agrícola": "max",
    "Pastagem com Manejo": "min",
    "Vegetação Florestal": "max",
    "Mosaico de Ocupações em Área Florestal": "max",
    "Mosaico de Ocupações em Área Campestre": "max",
    "Área Descoberta": "min",
    "Área Artificial": "min",
    "Área Úmida": "max"
}

# Criar uma cópia do DataFrame para aplicar a normalização sem alterar o original
df_norm = df.copy()

# Instanciar o normalizador Min-Max para ajustar os valores entre 0 e 1
scaler = MinMaxScaler()

# Aplicar a normalização para todas as colunas dos critérios
# Isso converte os valores absolutos (em km²) para uma escala de 0 a 1
df_norm[criterios] = scaler.fit_transform(df[criterios])

# Calcular a fuzzy membership (grau de pertinência, µ) para cada critério:
# Se o objetivo é maximizar, usamos o valor normalizado.
# Se o objetivo é minimizar, usamos o complemento (1 - valor normalizado).
for crit in criterios:
    if objetivos[crit] == "max":
        # Para critérios a maximizar, o valor normalizado é o grau de pertinência
        df_norm[crit + "_mu"] = df_norm[crit]
    else:
        # Para critérios a minimizar, definimos µ como 1 - valor normalizado
        df_norm[crit + "_mu"] = 1 - df_norm[crit]
        
# Definir a não-pertinência (ν) e hesitação (π) de forma simples:
# Aqui, ν é o complemento de µ, e π é definido como 0 (sem incerteza adicional)
for crit in criterios:
    df_norm[crit + "_nu"] = 1 - df_norm[crit + "_mu"]
    df_norm[crit + "_pi"] = 0  # Poderíamos ajustar se tivéssemos uma medida de incerteza

# Agregar os valores fuzzy em um índice global.
# Neste exemplo, calculamos a média simples dos graus de pertinência (µ) para todos os critérios.
# Esse índice global serve como o "fuzzy score" da alternativa.
df_norm["Fuzzy_Score"] = df_norm[[crit + "_mu" for crit in criterios]].mean(axis=1)

# Ordenar os dados de forma decrescente, de modo que as alternativas com maior fuzzy score apareçam primeiro
df_norm = df_norm.sort_values(by="Fuzzy_Score", ascending=False)

# Selecionar as colunas que serão exibidas no resultado final:
# "Estado", "Ano", o fuzzy score global e os valores µ para cada critério
colunas_resultado = ["Estado", "Ano", "Fuzzy_Score"] + [crit + "_mu" for crit in criterios]
resultado = df_norm[colunas_resultado]

# Salvar o resultado final em um CSV para análise posterior
resultado.to_csv("fuzzy_model_resultado.csv", index=False)

# Imprimir uma mensagem de confirmação e exibir as primeiras linhas do resultado
print("✅ Resultado do modelo fuzzy salvo em fuzzy_model_resultado.csv")
print(resultado.head())


✅ Resultado do modelo fuzzy salvo em fuzzy_model_resultado.csv
   Estado   Ano  Fuzzy_Score  Área Agrícola_mu  Pastagem com Manejo_mu  \
35     BR  2000     0.614753          0.666241                0.218208   
41     BR  2020     0.613741          1.000000                0.000000   
39     BR  2016     0.609975          0.934488                0.011766   
40     BR  2018     0.609237          0.964993                0.006200   
38     BR  2014     0.605333          0.906709                0.007410   

    Vegetação Florestal_mu  Mosaico de Ocupações em Área Florestal_mu  \
35                1.000000                                   1.000000   
41                0.920615                                   0.972096   
39                0.925846                                   0.994539   
40                0.923918                                   0.981723   
38                0.931974                                   0.975747   

    Mosaico de Ocupações em Área Campestre_mu  Área D